### CHATGPT api 활용: 프롬프트 엔지니어링

#### 연결 테스트

In [1]:
! pip install openai

   ---------------------------------------- 0.0/360.4 kB ? eta -:--:--
   ---------------------------------------- 360.4/360.4 kB 7.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
   ---------------------------------------- 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/77.9 kB ? eta -:--:--
   ---------------------------------------- 77.9/77.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/191.0 kB ? eta -:--:--
   ---------------------------------------- 191.0/191.0 kB 5.6 MB/s eta 0:00:00


In [1]:
from openai import OpenAI

In [2]:
# 개인 키 발급
OPENAI_API_KEY = "  "

In [3]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "안녕!!!!"}
    ],
    temperature = 0.7,
    top_p=0.3
)

In [5]:
completion

ChatCompletion(id='chatcmpl-9towrDCDrDxXJ8Pd99G8Hzs2qk9ho', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요! 어떻게 도와드릴까요?', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723090913, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_48196bc67a', usage=CompletionUsage(completion_tokens=10, prompt_tokens=10, total_tokens=20))

In [6]:
results = completion.choices[0].message.content
print(results)

안녕하세요! 어떻게 도와드릴까요?


In [ ]:
# 적용 가능한 파라미터   
# temperature=0.2 -- 0~2.0 사이 (2.0 일수록 다양한 답변 생성)
# top_p=0.5 -- 0~1 사이 (0일수록 일반적으로 예측할 수 있는 답변 생성)
# max_tokens=1000 # 최대 토근 수 지정 gpt-3.5-turbo = 4096 max

 
1. '문장' 단위 첨삭 필요 (chat gpt 단독 사용이 아닌, 외부 시스템 개발을 위해서임)
2. 독후감 내 글쓴이가 '느낀점'(감정) 이 있는 문장을 찾기 (줄거리에 대한 내용은 제외)  
3. 느낀점이 있는 문장에 대해서는 감정 '키워드'를 찾아야함 (다중 키워드도 가능)
4. [독후감]  
"주인공 은석이는 학교를 다닌다. 나는 은석이가 배가 고파서 굶을때마다 너무 마음이 아팠다. 은석이가 꼭 행복해지길 빈다."  
문장1: "주인공 은석이는 학교를 다닌다." -> "없음"  
문장2: "나는 은석이가 배가 고파서 굶을때마다 너무 마음이 아팠다." -> "슬픔"   
문장3: "은석이가 꼭 행복해지길 빈다." -> "행복, 소망"
5. 문장 별로 개선 사항을 제시함
6. 결과에 포함될 내용: 문장 index, 문장, 해당 문장에 존재하는 감정 키워드, 개선 내용 제안 -> 포맷은 자유  

### 개요 

- 목적 : 주어진 독후감 텍스트에서 글쓴이의 '느낀점'이 포함된 문장을 식별하고, 각 문장에 포함된 감정 '키워드'를 추출하며, 각 문장에 대한 개선 사항을 제시하는 프로그램 개발
##### 프로그램 설계
- 입력 데이터 : 독후감 3개를 texts 변수에 저장 (제목과 함께 저장)
- 결과 저장 : results 리스트에 저장
- 반복 처리 : texts 내용을 user_prompt로 설정하여 OpenAI API를 통해 시스템 프롬프트와 유저 프롬프트를 사용해 채팅 컴플리션 생성
- 결과 출력 : 3개의 독후감 첨삭 결과 출력

In [18]:
texts = [
    ("미래가 온다, 뇌과학", "이 책은 뇌에 대해 알 수있는 책이다. 짧고 간략한 내용이지만 톡톡 튀는 재미도 느끼며 수많은 뇌의 정보와 뇌의 역할, 뇌가 어떻게 이루어져 있는지 다 알수있다! 독자들은 이 책을 읽으면서 뇌의 신비함과 여러가지 과학잡식들을 배우 면서 느낄 수있다. 나는 이 책을 읽으면서 뉴런과 피걸에 대해 알 수있었다. 어떤가,흥미롭지 않은가? 뉴런은 뇌에 들어있는 세포들이다. 뉴런은 연합뉴런,신경뉴런,운동뉴런이 있다. 7개의 뉴런은 1000개의 또다른 뉴런들과 연결될 수 있다.이 뉴런들이 서로 연결 될 수있는 신경고리가 있는데 이 신경고리를 커넥텀이라 한다. 사람마다 서로 생각,느낌,감정들이 다른 이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문이다. 또한 사람들이 서로의 경험이 다르고 커넥톰이 다른 것도 한몫한다. 동물들에 따라 뇌에 있는 피질이 있는데 이 피질이 동물의 기억력을 좌지우지 한다.개구리,물고기,악어의 피질은 거의 뇌가 매끈매끈하기 때문에 거의 없다. 그래서 그들은 장기기억력이 없어 지금밖에 기억을 할 수 없다.하지만 쥐는 조금 더 뇌가 크기 때문에 전에 있었던 일을 기억할 수 있다. 하지만사람에 비하면 아무것도 아니다.사람은 뇌가 쭈굴쭈굴 주름졌다.따라서 우리는 장기기억력이 상당히 다른동물에 비하면 높고 미래를 생각하고 행동하며 상상할 수 있다.이렇게 많은 내용을 알 수 있으니 나는 이책을 정말 재밌게 읽었다. "),
    ("와이파이를 먹어버린 펄럭버지들", "나는 이 책을 읽고 인터넷 예절이 왜 이렇게까지 중요한지 다시 한번 알게 되었다. 이 책에서 나온 것처럼 동방삭은 형준이의 핸드폰을 훔쳐 와 인터넷 예절을 지키지 않고 핸드폰을 사용해 다른 사람들에게 많은 피해를 주었다.   우선 형준이 ID로 게임 쇼핑을 해서 형준이가 한 것도 아닌데 형준이가 엄마에게 혼났고, PC방에서 형준이 ID로 선생님 컴퓨터를 해킹해서 자료를 모두 삭제해버렸다. 이런 일들 때문에 형준이는 자기 잘못도 아닌데 오해를 받아야 했고, 선생님조차도 피해를 봤다. 또, 불법 영화를 다운로드해서 영화를 만든 사람들에게 피해를 주었다. 또, 형준이 ID로 인터넷에 거짓 뉴스를 올리기도 했다. 바로 가수 ‘민’이 콘서트를 한다는 거짓 뉴스였다. 그래서 동방삭은 가수 민의 명예도 훼손하고 말았다.   사실 이러한 일들은 현실에서도 빈번하게 일어난다. 특히 연예인들은 자신의 악성 댓글을 보고 너무 힘들어 자살했다는 소식도 어렵지 않게 접할 수 있다. 자기 자신에게는 ‘장난’일 수 있다. 하지만 다른 사람한테는 피해를 줄 수 있다. 만약 악성 댓글을 쓴 사람을 찾더라도 이미 상처를 받은 뒤다. 그래서 그 고통이 쉽게 잊히지 않을 수도 있다. 또 연예인들은 악성 댓글 때문에 활동을 중지하거나 그룹에서 탈퇴하기도 한다. 이처럼 댓글 한 줄이라는 조그만 일이 큰일을 만들 수도 있는 것이다.   더불어 게임 머니를 살 때 부모님의 주민등록번호를 몰래 이용하는 경우도 꽤 있다. 이런 일들은 초등학생들에게 많이 일어난다. 이렇게 허락도 없이 부모님의 주민등록번호를 이용하면 부모님은 해킹을 당했다고 생각하고 경찰에 신고를 할 수도 있다. 자기는 게임 머니 하나 사는 것쯤은 괜찮을 수도 있지만 부모님께서는 누가 게임 머니를 샀는지 모르니 큰 일이라고 생각 할 수도 있으므로 게임 머니를 살 때는 부모님에게 허락을 받고 사야한다.   그러므로 인터넷에서는 더욱더 예절을 지켜야한다. 대개 사람들은 자신의 모습이 드러나는 곳에서는 예의 바르게 행동하고, 예절을 잘 지켜야 한다고 생각한다. 그러나 인터넷에서는 자신의 모습이 드러나지 않는다고 악성 댓글을 남기거나 해킹을 해도 괜찮다고 생각한다. 그래서 사람들이 피해를 보고 상처를 받는 것이다.악성 댓글을 남기고 해킹을 하더라도 쉽게 추적할 수 있다고 한다. 그래서 나는 더욱이 인터넷 예절을 지키지 않는 사람들이 이해가 가지 않는다. 설사 자기 마음에 들지 않는다면 그저 지나치면 되는데 말이다. 나는 아직 카톡 말고는 인터넷을 많이 사용하지 않는다. 카톡을 하는데도 예의를 지키지 않는 사람들이 꽤 있다나는 이야기를 들었다. 나는 내 주변에 가족이나 친구들이 카톡에서든 인터넷에서 예의를 지키지 않는 것을 목격한다면 인터넷 예의를 지키지 않으면 다른사람에게 피해를 주니 예의를 지켜서 인터넷을 사용하라고 얘기해줄 것이다. 따라서 자신의 이름과 얼굴이 드러나는 곳에서만 예의를 지키지 말고, 인터넷에서도 자신의 모습이 드러난다고 생각하고 예의를 지켜야 한다고 생각한다. 인터넷 예절을 지키지 않는 사람들은 ‘한 번 이런다고 걸리겠어?’라는 마음일 것이다. 마치 길거리에 쓰레기를 버리는 사람들처럼 말이다. 하지만 ‘내가 인터넷 예절을 지키지 않으면 다른 사람에게 피해를 줄 수 있으니 나 하나라도 인터넷 예절을 잘 지켜야겠어’라는 생각을 가졌으면 좋겠다.  "),
    ("책과 노니는 집", " '책과 노니는집은 아버지가 천주교를 믿는것으로 의심 받아서 결국 돌아가셨지만 슬픔을 이겨낸 장이의 이야기다 아버지는 최서쾨의 부탁으로 천주교의 책을 배끼지만 그만걸려서 심한매질을 당하다가 집으로 겨우 살아서 돌아왔다 하지만 온몸에 상처에 멍 에 온갖 상처가 나고 결국 돌아가버리셨다 장이는 아버지를 잃어 슬플 지 만 아버지에 꿈을 이루기 위해 최서쾌의 책방에서 책심부름든 여러 신부름을 하다 책을전달 했는데 그책이 천주교책이여서 오해를 받아 포졸들 이처들어 오지만 현명 하게 돈망쳐서살아나마다 결국 아버지의 소월 이 책방을 얻어서 아버지의 꿈을 이루 웠다인상깊은 장면은 아버지가 책을 배끼다가 천주교를 믿는다고 오해를받아서 돌아가신장면 이장면에서 아무죄 없는 사람들이 죽는다느게 정말 속상하고,억울하고,슬푸거 같다 조선시대에 천주교를 막게 하는법이 사라지면 좋겠다 사람들이 천주교를 믿었다는 이유로 처영 되거나 감옥에 들어가 심한 매질을 당하기도하는대 겨우 천주교를 믿는다고 목숨을 잃거나 심한고문 을 당하는게 정말 안타갑다 만약에 없다면 사람들이 목숨을 없어질 일은 없었을 것이다 장이가 부디 행복하고 이제 쫒이지 않고 자유롭게 책을배달하고 팔고 책을읽어서 부디 훌륭한 사람이 되면 좋을거같다 그동안 아버지를 잃고, 쫒기고,울고,등등 정말 마음이 아파서 부디 이제는 행복한고,즐검고,행복한 생각만 하고 아이들처럼 즐기면 좋겠다. 책방이 성공해서 부자가 되었으면 좋겠다.")
]

In [23]:
# 프롬프트 작성
system_prompt = """
주어진 독후감 텍스트에서 글쓴이가 '느낀점'이 있는 문장을 찾고, 
각 문장에 포함된 감정 '키워드'를 도출합니다. 
또한, 각 문장 별로 개선 사항을 제시합니다. 
줄거리에 대한 문장은 제외하고, 감정이 포함된 문장만을 대상으로 합니다.

결과는 다음의 형식으로 출력:
문장: {문장 번호}
문장: "{문장 내용}"
해당 문장에 존재하는 감정 키워드: "{감정 키워드}"
개선 내용 제안: "{개선 내용}"
예시 입력: "주인공 은석이는 학교를 다닌다. 나는 은석이가 배가 고파서 굶을 때마다 너무 마음이 아팠다. 은석이가 꼭 행복해지길 빈다."
예시 출력:  
문장1: "주인공 은석이는 학교를 다닌다." -> "없음"
문장2: "나는 은석이가 배가 고파서 굶을 때마다 너무 마음이 아팠다." -> "슬픔"
개선 내용 제안: 은석이의 배고픔을 좀 더 구체적으로 묘사하여 독자가 더 공감할 수 있게 합니다. 예: "나는 은석이가 배가 고파서 굶을 때마다 가슴이 아리고, 눈물이 날 것 같았다."
"""


In [25]:
results = []  # Initialize results as an empty list

for idx, (title, content) in enumerate(texts):
    user_prompt = content.strip()
    
    # Create a completion with the API
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # Extract the result from the completion
    result = completion.choices[0].message.content
    
    # Append the result to the list
    results.append(f"{title} 결과:\n{result}\n")

# Print all results
for result in results:
    print(result)
    print("-------------------------------------------------------------------")

미래가 온다, 뇌과학 결과:
문장1: "독자들은 이 책을 읽으면서 뇌의 신비함과 여러가지 과학잡식들을 배우면서 느낄 수있다."
해당 문장에 존재하는 감정 키워드: "신비함, 호기심"
개선 내용 제안: 독자들이 신비함을 느낄 수 있는 구체적인 예시를 포함하여 감정 이입을 유도합니다. 예를 들어, "독자들은 이 책을 읽으면서 뇌의 신비함과 그러한 신비의 이면에 숨겨진 과학 잡식들을 배우며, 마치 미지의 세계를 탐험하는 듯한 호기심을 느낄 수 있다." 라고 서술할 수 있습니다.

문장2: "나는 이 책을 읽으면서 뉴런과 피걸에 대해 알 수있었다."
해당 문장에 존재하는 감정 키워드: "배움, 흥미"
개선 내용 제안: 배움의 기쁨을 더 강조하기 위해 "정말로 흥미롭게 느낄 만큼"과 같은 표현을 추가하여 감정을 강화합니다. 예를 들어, "나는 이 책을 읽으면서 뉴런과 피질에 대해 알게 되었고, 그 과정이 정말 흥미롭다고 느꼈다."라고 명확히 서술할 수 있습니다.

문장3: "이렇게 많은 내용을 알 수 있으니 나는 이 책을 정말 재밌게 읽었다."
해당 문장에 존재하는 감정 키워드: "재미, 만족"
개선 내용 제안: 독자가 왜 재미를 느꼈는지 그 이유를 더 구체적으로 표현하여 감정을 더욱 풍부하게 전달합니다. 예를 들어, "이렇게 다양한 뇌에 대한 내용을 알 수 있으니 나는 이 책을 정말 재밌게 읽었고, 그 지식이 내 생각을 넓혀 주는 것 같아 매우 만족스러웠다."라고 후기를 보다 풍성하게 할 수 있습니다.

-------------------------------------------------------------------
와이파이를 먹어버린 펄럭버지들 결과:
문장1: "나는 이 책을 읽고 인터넷 예절이 왜 이렇게까지 중요한지 다시 한번 알게 되었다."  
해당 문장에 존재하는 감정 키워드: "깨달음"  
개선 내용 제안: "책을 읽고 인터넷 예절의 중요성을 깨달았다는 부분을 보다 감성적으로 표현하여 독자의 공감을 이끌어낼 수 있습니다." 예: "이 책을 읽고 인터넷